## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [134]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [135]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [136]:
# Set up data path
data_path = 'data'

In [139]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,10/4/16,Citibike to Work,1.36,NaN,11.0,0.18,7.42,Citibike
1,8/17/16,Citibike from Work,0.96,NaN,10.0,0.17,5.76,Citibike
2,4/7/20,CP BL x 2,13.98,1.0,43.0,1.72,8.14,Specialized
3,9/9/19,To Essex Market,3.71,NaN,31.0,0.52,7.18,Citibike
4,12/2/16,Citibike to Work,1.17,NaN,11.0,0.18,6.38,Citibike
...,...,...,...,...,...,...,...,...
348,10/11/16,Citibike to Work,1.33,NaN,11.0,0.18,7.25,Citibike
349,6/27/20,Wards Island,13.80,1.0,49.0,1.82,7.60,Specialized
350,10/8/20,Wards Island x 2,17.71,2.0,7.0,2.12,8.37,Specialized
351,9/6/19,CP BL,7.44,NaN,57.0,0.95,7.83,Specialized


In [140]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [141]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,1.36,0.18,7.42,Citibike
1,0.96,0.17,5.76,Citibike
2,13.98,1.72,8.14,Specialized
3,3.71,0.52,7.18,Citibike
4,1.17,0.18,6.38,Citibike
...,...,...,...,...
348,1.33,0.18,7.25,Citibike
349,13.80,1.82,7.60,Specialized
350,17.71,2.12,8.37,Specialized
351,7.44,0.95,7.83,Specialized


In [142]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
#new2_df = all_df[['Miles', 'Duration', 'Speed', 'Type']]
#new2_df

In [143]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [144]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [145]:
# Checking training data shape
X_train.shape

(264, 3)

In [146]:
# Checking testing data shape
X_test.shape

(89, 3)

In [147]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [148]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [149]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [150]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

1.0


In [166]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Specialized
1,Citibike
2,Specialized
3,Specialized
4,Specialized
...,...
84,Citibike
85,Citibike
86,Specialized
87,Citibike


In [167]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
236,Specialized
293,Citibike
124,Specialized
148,Specialized
102,Specialized
...,...
317,Citibike
218,Citibike
59,Specialized
163,Citibike


In [168]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,236,Specialized
1,293,Citibike
2,124,Specialized
3,148,Specialized
4,102,Specialized
...,...,...
84,317,Citibike
85,218,Citibike
86,59,Specialized
87,163,Citibike


### Combining the test and prediction dataframes for comparison

In [169]:
# Combining the new test and predition dataframes horizontally
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,236,Specialized,Specialized
1,293,Citibike,Citibike
2,124,Specialized,Specialized
3,148,Specialized,Specialized
4,102,Specialized,Specialized
...,...,...,...
84,317,Citibike,Citibike
85,218,Citibike,Citibike
86,59,Specialized,Specialized
87,163,Citibike,Citibike


In [170]:
# Exporting Combined Test and Prediction Dataframe for examination
test_pred_file = "test_pred.csv"
test_pred_df.to_csv(os.path.join(data_path, test_pred_file))